In [1]:
import rospy
import jupyros as jr
from nav_msgs.msg import Odometry
import actionlib
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from assignment_2_2023.msg import PlanningAction, PlanningGoal, Vel

In [2]:
# Initialize ROS node
rospy.init_node('Jupyter3')

In [3]:
# Global variables
achieved_goals_count = 0
cancelled_goals_count = 0
sent_goals_count = 0
x_goal, y_goal = (0, 0)
new_goal_flag = False
sent_goals_list = []
cancelled_goals_list = []
achieved_goals_list = []
active_goal = False  # Flag to track if there is an active goal

In [4]:
# Set up the action client
client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
client.wait_for_server()

True

In [5]:
# Function to send goal to action server
def send_goal(x, y):
    global x_goal, y_goal
    goal = PlanningGoal()
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y
    client.send_goal(goal, feedback_cb=action_feedback_callback)
    sent_goal_callback(goal)

# Function to cancel the current goal
def cancel_goal():
    global active_goal
    if active_goal:  # Only cancel if there is an active goal
        client.cancel_goal()
        cancelled_goal_callback()

In [6]:
# Create widgets for inputting x and y coordinates and buttons to send/cancel the goal
x_input = widgets.FloatText(description='X Goal:')
y_input = widgets.FloatText(description='Y Goal:')
send_goal_button = widgets.Button(description='Send Goal')
cancel_goal_button = widgets.Button(description='Cancel Goal')

In [7]:
# Define the button click event for sending goal
def on_send_goal_button_clicked(b):
    send_goal(x_input.value, y_input.value)

# Define the button click event for cancelling goal
def on_cancel_goal_button_clicked(b):
    cancel_goal()

send_goal_button.on_click(on_send_goal_button_clicked)
cancel_goal_button.on_click(on_cancel_goal_button_clicked)

In [8]:
# Display the widgets
display(widgets.VBox([x_input, y_input, send_goal_button, cancel_goal_button]))

[INFO] [1717016376.069257, 4506.668000]: Goal cancelled


In [9]:
# Display current position widgets
current_position_label = widgets.Label(value='Current Pose')
display(current_position_label)
current_x_widget = widgets.FloatText(value=0, description='x:', disabled=True)
display(current_x_widget)
current_y_widget = widgets.FloatText(value=0, description='y:', disabled=True)
display(current_y_widget)

Label(value='Current Pose')

FloatText(value=0.0, description='x:', disabled=True)

FloatText(value=0.0, description='y:', disabled=True)

In [10]:
# Widgets for displaying goals and statuses
sent_goals_display = widgets.HTML(value='')
cancelled_goals_display = widgets.HTML(value='')
achieved_goals_display = widgets.HTML(value='')

In [11]:
def action_feedback_callback(feedback):
    global achieved_goals_count, achieved_goals_list, achieved_goals_display, active_goal
    if feedback.stat.strip() == 'Target reached!':
        achieved_goals_count += 1
        achieved_goals_list.append((x_goal, y_goal))
        achieved_goals_display.value = "<br>".join(
            f"Pose {i + 1}: ({achieved_goals_list[i][0]}, {achieved_goals_list[i][1]})"
            for i in range(len(achieved_goals_list))
        )
        active_goal = False  # Goal is achieved, no active goal

def cancelled_goal_callback():
    global cancelled_goals_count, active_goal
    if active_goal:  # Only cancel if there is an active goal
        rospy.loginfo('Goal cancelled')
        cancelled_goals_count += 1
        cancelled_goals_list.append((x_goal, y_goal))
        cancelled_goals_display.value = "<br>".join(
            f"Pose {i + 1}: ({cancelled_goals_list[i][0]}, {cancelled_goals_list[i][1]})"
            for i in range(len(cancelled_goals_list))
        )
        active_goal = False  # Goal is canceled, no active goal

def sent_goal_callback(goal):
    global x_goal, y_goal, new_goal_flag, sent_goals_count, sent_goals_list, sent_goals_display, active_goal
    x_goal = goal.target_pose.pose.position.x
    y_goal = goal.target_pose.pose.position.y
    new_goal_flag = True
    sent_goals_count += 1
    sent_goals_list.append((x_goal, y_goal))
    sent_goals_display.value = "<br>".join(
        f"Pose {i + 1}: ({sent_goals_list[i][0]}, {sent_goals_list[i][1]})"
        for i in range(len(sent_goals_list))
    )
    active_goal = True  # New goal is sent, active goal

In [12]:
# Visualization class for live plotting
class GoalVisualizer:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.ln_goal, = plt.plot([], [], 'bs')
        self.x_data, self.y_data = [], []
        self.x_goals, self.y_goals = [], []
    
    def initialize_plot(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln, self.ln_goal
    
    def pose_callback(self, pose_msg):
        self.update_pose_data(pose_msg)
        self.update_goal_data()
        self.update_current_location()
    
    def update_pose_data(self, pose_msg):
        x = pose_msg.pose.pose.position.x
        y = pose_msg.pose.pose.position.y
        self.x_data.append(x)
        self.y_data.append(y)
    
    def update_goal_data(self):
        global new_goal_flag, sent_goals_display
        if new_goal_flag:
            self.x_goals.append(x_goal)
            self.y_goals.append(y_goal)
            new_goal_flag = False
    
    def update_current_location(self):
        global current_x_widget, current_y_widget
        current_x_widget.value = self.x_data[-1]
        current_y_widget.value = self.y_data[-1]
    
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        try:
            self.ln_goal.set_data(self.x_goals[-1], self.y_goals[-1])
        except IndexError:
            pass
        return self.ln, self.ln_goal

In [13]:
# Initialize visualization
%matplotlib widget
visualizer = GoalVisualizer()
rospy.Subscriber("/odom", Odometry, callback=visualizer.pose_callback)
animation = FuncAnimation(visualizer.fig, visualizer.update_plot, init_func=visualizer.initialize_plot, blit=True, interval=100)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Display goal statuses
sent_goal_label = widgets.Label(value='Sent Goals:')
display(sent_goal_label)
display(sent_goals_display)

achieved_goal_label = widgets.Label(value='Achieved Goals:')
display(achieved_goal_label)
display(achieved_goals_display)

cancelled_goal_label = widgets.Label(value='Cancelled Goals:')
display(cancelled_goal_label)
display(cancelled_goals_display)

Label(value='Sent Goals:')

HTML(value='')

Label(value='Achieved Goals:')

HTML(value='')

Label(value='Cancelled Goals:')

HTML(value='')

In [15]:
# Plot goal statistics
fig, ax = plt.subplots()
def update_goal_statistics(frame):
    xlabels = ['Sent', 'Achieved', 'Cancelled']
    count = [len(sent_goals_list), len(achieved_goals_list), len(cancelled_goals_list)]
    ax.cla()
    ax.bar(xlabels, count, color=['blue', 'green', 'red'], width=0.6)
    plt.show()

statistics_animation = FuncAnimation(fig, update_goal_statistics)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …